# Tutorial 4: Nodes, Synchronization and Graph Validity

In this tutorial, we will discuss [nodes](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html) and the validity of the [graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html).

The following will be covered:
- Creating a node
- Adding a node to the graph
- Checking the validity of the graph
- Synchronization within EAGERx

In the remainder of this tutorial we will go more into detail on these concepts.

Furthermore, at the end of this notebook you will find an assignment.
For the assignment you will have to add/modify a couple of lines of code, which are marked by

```python

# START ASSIGNMENT [BLOCK_NUMBER]

# END ASSIGNMENT [BLOCK_NUMBER]
```

## Pendulum Swing-up

We will create an environment for solving the classic control problem of swinging up an underactuated pendulum, very similar to the [Pendulum-v0 environment](https://gym.openai.com/envs/Pendulum-v0/).
Our goal is to swing up this pendulum to the upright position and keep it there, while minimizing the velocity of the pendulum and the input voltage.

Since the dynamics of a pendulum actuated by a DC motor are well known, we can simulate the pendulum by integrating the corresponding ordinary differential equations (ODEs):


$\mathbf{x} = \begin{bmatrix} \theta \\ \dot{\theta} \end{bmatrix} \\ \dot{\mathbf{x}} = \begin{bmatrix} \dot{\theta} \\ \frac{1}{J}(\frac{K}{R}u - mgl \sin{\theta} - b \dot{\theta} - \frac{K^2}{R}\dot{\theta})\end{bmatrix}$

with $\theta$ the angle w.r.t. upright position, $\dot{\theta}$ the angular velocity, $u$ the input voltage, $J$ the inertia, $m$ the mass, $g$ the gravitational constant, $l$ the length of the pendulum, $b$ the motor viscous friction constant, $K$ the motor constant and $R$ the electric resistance.

## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package and ROS.

In [ ]:
try:
    import eagerx_tutorials
except ImportError:
    !{"pip install eagerx-tutorials  >> /tmp/eagerx_install.txt"}
if 'google.colab' in str(get_ipython()):
  !{"curl 'https://raw.githubusercontent.com/eager-dev/eagerx_tutorials/master/scripts/setup_colab.sh' > ~/setup_colab.sh"}
  !{"bash ~/setup_colab.sh"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()
env = None

# Allows reloading of registered entites from changed files
# Required in interactive notebooks only.
%reload_ext autoreload
%autoreload 1

## Let's get started

We start by importing the required packages and initializing EAGERx.

In [2]:
import eagerx
import eagerx_tutorials.pendulum  # Registers Pendulum
import eagerx_ode  # Registers OdeBridge

# Initialize eagerx (starts roscore if not already started.)
eagerx.initialize("eagerx_core")

... logging to /home/jelle/.ros/log/8f35a712-c6c7-11ec-85e9-af6a0749fbba/roslaunch-jelle-Alienware-m15-R4-96300.log
started roslaunch server http://145.94.217.36:34855/
ros_comm version 1.15.14


SUMMARY

PARAMETERS
 * /rosdistro: noetic
 * /rosversion: 1.15.14

NODES

[INFO] [1651155597.413962]: Roscore cannot run as another roscore/master is already running. Continuing without re-initializing the roscore.


In this tutorial we will discuss the entity type [nodes](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html).
In the previous tutorials we have showed how to add [object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html) entities to a [graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html).
Apart from objects, we can also add nodes to a graph.
The difference between nodes and objects, is that nodes are bridge-agnostic while objects have bridge-specific implementations.
Objects are entities that can sense or act in the environments.
Nodes on the other hand can only process data.
A node could for example contain a classifier, PID controller or filter.
While objects have sensors and actuators, a node has inputs and outputs.

In this tutorial we will create a moving average filter that will act as a low-pass filter on the voltage $u$.
This will fil

In [ ]:
%%writefile node.py

from typing import Optional

# IMPORT ROS
from std_msgs.msg import Float32MultiArray

# IMPORT EAGERX
import eagerx.core.register as register
from eagerx.utils.utils import Msg
from eagerx.core.entities import Node, Processor
from eagerx.core.constants import process


class MovingAverageFilter(Node):
    @staticmethod
    @register.spec("ExampleNode", Node)
    def spec(
        spec,
        name: str,
        rate: float,
        index: int = 0,
        process: Optional[int] = process.ENVIRONMENT,
    ):
        """
        MovingAverage filter
        :param spec: Not provided by user.
        :param name: Node name
        :param rate: Rate at which callback is called.
        :param index: Index (related to Float32MultiArray.data[index])
        :param process: {0: NEW_PROCESS, 1: ENVIRONMENT, 2: BRIDGE, 3: EXTERNAL}
        :return:
        """
        # Performs all the steps to fill-in the params with registered info about all functions.
        spec.initialize(MovingAverageFilter)

        # Modify default node params
        spec.config.name = name
        spec.config.rate = rate
        spec.config.process = process
        spec.config.inputs = ["signal"]
        spec.config.outputs = ["filtered"]

        # Add converter & space_converter
        spec.inputs.signal.converter = Processor.make("GetIndex_Float32MultiArray", index=index)
        spec.inputs.signal.space_converter = SpaceConverter.make("Space_Float32MultiArray", [-3], [3], dtype="float32")

    def initialize(self):
        self.average = 0.

    @register.states()
    def reset(self):
        self.average = 0.
        self.n = 0

    @register.inputs(signal=Float32MultiArray)
    @register.outputs(filtered=Float32MultiArray)
    def callback(self, t_n: float, signal: Optional[Msg] = None):
        self.n += 1
        data = signal.msgs[-1].data
        self.average = [(self.average * (self.n - 1) + data[0]) / self.n]
        return dict(filtered=Float32MultiArray(data=self.average))

In [ ]:
%aimport node
import MovingAverageFilter
